# RetSynth Software 
### RetSynth is a tool, developed with python, which identfies enzyme/reaction pairs that are required in a user specified microbial organisms to produce a target chemical compound. 

## Purpose
### In the process of bioengineering a microbial organism, identifying reactions/enzymes to transform into an organism for optimal production of a target compound is extremely difficult due to the vast number of reactions/enzymes in numerous organisms that maybe suitable for optimal production.  



In [ ]:
#Load in necessary libraries
from __future__ import print_function

from multiprocessing import Process
import argparse

from IPython.display import HTML

try:
    import pickle
except:
    import cPickle as pickle
import os
import re
import glob
import time
import shutil
import zipfile
from sys import platform
PATH = os.path.dirname(os.path.abspath('__file__'))

def verbose_print(verbose, line):
    if verbose:
        print(line)

def unzip_necessary_files_and_libraries():
  '''Unzip Default Databases and indigo libraries'''
  def unzip_folder(foldername):
      if os.path.isdir(foldername) is False:
        try:
            zipref = zipfile.ZipFile(foldername+'.zip', 'r')
            zipref.extractall('.')
        except:
            pass
            # print("WARNING: No "+foldername+".zip")
  try: 
      unzip_folder(PATH+"/ConstructedDatabases")
  except FileNotFoundError:
      print("WARNING: No ConstructedDatabase")

  if platform == 'darwin':
      unzip_folder(PATH+'/indigopython130_mac')

  elif platform == "linux" or platform == "linux2":
      unzip_folder(PATH+'/indigopython130_linux')

  elif platform == "win32" or platform == "win64" or platform == "cygwin":
      unzip_folder(PATH+'/indigopython130_win')

unzip_necessary_files_and_libraries()

from timeit import default_timer as timer
from rsgc.Parser import read_startcompounds as rtsc
from rsgc.Parser import read_targets as rt
from rsgc.Parser import generate_output as go
from rsgc.Parser import structure_similarity as ss
from rsgc.Parser import generate_html as gh
from rsgc.Visualization_chemdraw import reaction_files as rf
from rsgc.Visualization_graphviz import SP_Graph_dot as spgd
from rsgc.ShortestPath import extractinfo as ei
from rsgc.ShortestPath import constraints as co
from rsgc.ShortestPath import integerprogram_pulp as ip_pulp
from rsgc.Database import initialize_database as init_db
from rsgc.Database import build_modelseed as bms
from rsgc.Database import build_metacyc_db as bmcdb
from rsgc.Database import build_user_rxns_db as burdb
from rsgc.Database import build_ATLAS_db as batlasdb
from rsgc.Database import build_MINE_db as bminedb
from rsgc.Database import build_KEGG_db as bkeggdb
from rsgc.Database import build_SPRESI_db as bspresidb
from rsgc.Database import query as Q
from rsgc.Database import remove_duplicate_cpds as rdc
from rsgc.FBA import build_model as bm
from rsgc.FBA import optimize_target as ot
from rsgc.FBA import compare_results as cr
from rsgc.FBA import retrieve_producable_mets as rpm
from rsgc.FBA import compareKO_results as crko
from rsgc.GeneCompatibility.gc import gc_main as gc
from rsgc.GeneCompatibility.gc import gc_enzyme as gce

### Load in parameters for RetSynth Run 
#### Go through the parameters and make sure you specify a path to a target file (list of targets and host organisms, targets_path option as well as generate a database (generate_database & generate_database_constraints) or use pre-constructed database with database and database_constraint options. If you are generating a new constraint file make sure to read the documentation and specify necessary database construction parameters. 

In [ ]:
#Load Parameters for RetSynth Run 
targets_path="testtargets.txt"
output_path="Results/"
output_html=True
output_xlsx_format=False
processors=4
start_compounds=None
verbose=False

#database parameters
generate_database="database.db"
generate_database_constraints="database.constraints"
database=False
database_constraints=False
inchidb=True

#database construction parameters
patric_models=True
patric_username=None
patric_password=None
patric_reaction_type="bio"
patric_media="Carbon-D-Glucose"
patric_sbml_output=False
patricfile='PATRIC_genome_complete.csv'
patric_models_already_built=False
metacyc=True
metacyc_addition=""
metacyc_reaction_type="bio"
kegg=False
kegg_reaction_type="bio"
kegg_organism_type="bacteria"
kegg_number_of_organisms="all"
kegg_number_of_organism_pathway="all"
atlas=False
atlas_dump_directory=None
atlas_reaction_type="bio"
mine=False
mine_dump_directory=None
mine_reaction_type=True
SPRESI=False
spresi_dump_directory=None
spresi_reaction_type="chem"
user_rxns_2_database=False
user_rxns_2_database_type="bio"

#Flux balance analysis 
flux_balance_analysis=True
media_for_FBA="Carbon-D-Glucose"
knockouts=False

#Other parameters for RetSynth
limit_reactions=10
limit_cycles='None'
solver_time_limit=30
evaluate_reactions="all" 
k_number_of_paths=0
multiple_solutions=str(True)
cycles=str(True)
run_tanimoto_threshold=False
tanimoto_threshold=1
figures_chemdraw=False
figures_graphviz=True
show_rxn_info=False
images=True
timer_output=False

##Gene compatibility parameters
gene_compatability=True
cai_threshold=0.23
user_cai_table=None

### Load in necessary functions for analysis

In [ ]:
#load necessary functions needed for downstream processing
def get_compartmentID_from_db(DB, compartment):
    '''Retrieves specified compartment ID'''
    compartment = compartment.lower()
    compartmentID_array = DB.get_compartment(compartment)
    if compartmentID_array is None or len(compartmentID_array) == 0 or compartmentID_array[0] == '':
        print ('WARNING: Could not retrieve a compartment ID from the database')
        if compartment == 'cytosol':
            compartmentID = 'c0'
        elif compartment == 'extracellular':
            compartmentID = 'e0'
        else:
            compartmentID = 'c0'
    else:
        compartmentID = compartmentID_array[0]
    return (compartmentID)

def get_new_temp_imgs_folder(PATH, count):
    '''Check if folder to store images is already present if so new temp folder is made'''
    count+=1
    try:
        os.mkdir(PATH+'/temp_imgs_'+str(count))
        return PATH+'/temp_imgs_'+str(count)
    except OSError:
        PATH_NEW = get_new_temp_imgs_folder(PATH, count)
        return PATH_NEW

def _specific_target(target_id):
    '''Determines if there was a specified organism'''

    if target_id in ['', 'NA', 'N/A']:
        return False

    else:
        return True

def targets_missing(targets,database):
    '''Check if targets are missing from the database'''
    DB = Q.Connector(database)
    all_compounds = DB.get_all_compounds()

    for t in targets:
        if t[0] not in all_compounds:
            return True
    return False

def run_flux_balance_analysis(target_info, ex_info, incpds_active,
                              inrxns, media, ko,
                              output, DB, verbose):
    '''
    Run flux balance analysis on target organism with added reactions
    necessary to produce target compound
    '''
    fba = bm.BuildModel(target_info[2], incpds_active, inrxns, DB, verbose, media)
    opt_fba = ot.OptimizeTarget(target_info[0], target_info[2], fba.model, ex_info.temp_rxns,
                                ex_info.temp_exmets, fba.compounds_dict, incpds_active,
                                inrxns, DB, verbose, ko)
    # print (opt_fba.fbasol.fluxes)
    comparisonresults = cr.Compare(target_info[0], fba.solution, opt_fba.fbasol,
                                   ex_info.temp_rxns, DB)
    output.output_FBA(target_info, fba.solution, opt_fba, comparisonresults, ex_info.temp_rxns)
    output.output_theoretical_yield(target_info[0], target_info[2], opt_fba.fbasol,
                                    opt_fba.compounds_dict)

    if ko:

        output.output_essential_reactions(target_info[0], target_info[2], opt_fba.essentialrxns)
        comparisonKOresults = crko.CompareKO(target_info[0], opt_fba.compounds_dict, opt_fba.fbasol,
                                             opt_fba.KOsolutions, ex_info.temp_rxns, DB)
        output.output_FBA_KOs(target_info, opt_fba.fbasol, opt_fba.compounds_dict, comparisonKOresults, ex_info.temp_rxns)

    return opt_fba


### Generate folder to store results

In [ ]:
#Generate new output folder
try:
    verbose_print(verbose, "STATUS: generating output folder "+output_path)
    os.mkdir(output_path)
except:
    verbose_print(verbose, "STATUS: output folder already exists "+output_path)
    pass

### Check and make sure all necessary arguments/options are specified 

In [ ]:
#Check arguments for errors
def check_arguments():
        '''Checks and makes sure all required arguments are provided'''
        if knockouts and not flux_balance_analysis:
            raise ValueError('--knockouts option requires that \
                        --flux_balance_analysis option be also specified')

        if metacyc and not metacyc_addition:
            raise ValueError('--metacyc requires use of parameters metacyc_addition')        

        if atlas and not atlas_dump_directory:
            raise ValueError('--atlas requires use of --atlas_dump_directory')

        if SPRESI and not spresi_dump_directory:
            raise ValueError('--SPRESI requires use of --spresi_dump_directory')

        if mine and not mine_dump_directory:
            raise ValueError('--mine requires use of --mine_dump_directory')

        if patric_models and not patric_password:
            raise ValueError('--patric_models requires options --patric_models, --patric_username, and --patric_password be specified')
        
        if patric_models and not patric_username:
            raise ValueError('--patric_models requires options --patric_models, --patric_username, and --patric_password be specified')
        
        if patric_username and not patric_password:
            raise ValueError('--patric_username requires options --patric_models, --patric_username, and --patric_password be specified')

        if not patric_models and patric_models_already_built:
            raise ValueError('--previously_built_patric_models requires options --patric_models, --patric_username, and --patric_password be specified')

        if start_compounds and flux_balance_analysis:
            raise ValueError('Flux balance cannot be performed on a set of starting compounds\
                        would need to use an organisms metabolism to simulate flux')

        if not multiple_solutions and k_number_of_paths:
            raise ValueError('Cannot find k_number_of_paths correctly \
                        without finding all multiple_solutions')
        if not targets_path:
            raise ValueError('Requires an input file of target compounds')
#Run check
check_arguments()

### Build or load database depending on whether you specified the generate_database option or database option 

In [ ]:
#Build Database
def retrieve_database_info(database, generate_database):
    '''
    Generates database or uses previously generated database.
    Can also add metacyc database to a Kbase metabolic database
    '''
    if generate_database:
        '''
        Generate a database
        '''
        init_db.Createdb(generate_database)
        database = generate_database
        new_db = True
        print (database)

    elif database:
        new_db = False
        database = database

    if patric_models:
        #Add PATRIC repository to database
        bms.BuildModelSeed(username=patric_username, password=patric_password, rxntype=patric_reaction_type,
                            inchidb=inchidb, DBpath=generate_database, output_folder=output_path, media=patric_media, 
                            patricfile=patricfile, newdb=new_db, sbml_output=patric_sbml_output,
                            previously_built_patric_models=patric_models_already_built)

    if metacyc:
        #Add metacyc repository to database
        bmcdb.Translate(database, metacyc_addition,
                        inchidb, metacyc_reaction_type, verbose)

    if kegg and (patric_models or kbase or metacyc):
        #Add kegg repository database
        BKD = bkeggdb.CompileKEGGIntoDB(database, kegg_organism_type,
                                        inchidb, processors, kegg_number_of_organisms,
                                        kegg_number_of_organism_pathways,
                                        kegg_reaction_type, True)

    elif kegg and not kbase and not patric_models and not metacyc:
        #Add kegg repository database
        print ('STATUS: Add only KEGG to RetSynth database')
        BKD = bkeggdb.CompileKEGGIntoDB(database, kegg_organism_type,
                                        inchidb, processors,
                                        kegg_number_of_organisms, kegg_number_of_organism_pathways,
                                        kegg_reaction_type, False)

    if user_rxns_2_database:
        #Add user identified reactions
        burdb.AddUserRxns2DB(database, user_rxns_2_database,
                            model_id='UserAdded', rxntype=user_rxns_2_database_type)
    if SPRESI:
        #Translate synthetic rdf files from SPRESI into database
        DB = Q.Connector(database)
        cytosol_compartmentID = get_compartmentID_from_db(DB, 'cytosol')
        bspresidb.RDF_Reader(spresi_dump_directory,
                            database,
                            spresi_reaction_type,
                            cytosol_compartmentID, processors)

    if mine:
        #Add MINE repository to database
        bminedb.BuildMINEdb(mine_dump_directory, database,
                            inchidb, mine_reaction_type)

    if atlas:
        #Add ATLAS repository to database
        batlasdb.build_atlas(atlas_dump_directory, database, inchidb,
                                processors, atlas_reaction_type)

    if inchidb and (patric_models or metacyc or kegg or SPRESI or mine or atlas):
        #Remove duplicate compounds from database
        rdc.OverlappingCpdIDs(database)

    ##IF DATABASE IS NOT SPECIFIED USE DEFUALT DATABASE IN ./ConstructedDatabases FOLDER##
    if not generate_database and not database:

        if media_for_FBA == 'Carbon-D-Glucose':

            print ('WARNING: No database specified using pre constructed database with media {}'.format(media_for_FBA))

            database = PATH+'/ConstructedDatabases/DBINCHIECOLIDH1_GL_MC.db'
            DB = Q.Connector(database)

        elif media_for_FBA == 'Complete':

            print ('WARNING: No database specified using pre constructed database with media {}'.format(media_for_FBA))

            database = PATH+'/ConstructedDatabases/DBINCHIECOLIDH1_CP_MC_cas_SPRESI_reduced1x.db'            

            DB = Q.Connector(database)
    else:
        DB = Q.Connector(database)

    ###GET TYPE OF REACTIONS TO BE USED IN THE ANALYSIS SO CORRECT CONTRAINT FILE IS GENERATED##
    print (database)
    allcpds = DB.get_all_compounds()
    if evaluate_reactions == 'all':
        allrxns = DB.get_all_reactions()
    elif evaluate_reactions == 'bio':
        allrxns = DB.get_reactions_based_on_type('bio')
    elif evaluate_reactions == 'chem':
        allrxns = DB.get_reactions_based_on_type('chem')
    return(allcpds, allrxns, database)

#Run function

all_db_compounds, all_db_reactions, database = retrieve_database_info(database, generate_database)

### Read in targets from target_path as well build and load constraint object which is used to identify pathways for targets

In [ ]:
#Read in target file and constraints files
def read_in_and_generate_output_files(database, targets_path, run_tanimoto_threshold):
    '''Read in target input file and generate output files'''
    DB = Q.Connector(database)
    R = rt.Readfile(targets_path, DB, inchidb)
    if not R.targets:
        raise ValueError('ERROR:\tNo targets, try different compounds')
    try:
        verbose_print(verbose, "STATUS: generating output folder "+output_path)
        os.mkdir(output_path)
    except:
        verbose_print(verbose, "STATUS: output folder already exists "+output_path)
        pass
    DB_missing_targets = targets_missing(R.targets, database)
    if DB_missing_targets:
        run_tanimoto_threshold = True
        verbose_print(verbose, 'STATUS:\tTarget not found in database. Trying to find structurally similar compounds')
    temp_imgs_PATH = get_new_temp_imgs_folder(output_path, 0)
    OUTPUT = go.Output(DB, output_path, verbose, flux_balance_analysis, knockouts, timer_output, GC=gene_compatability)
    if run_tanimoto_threshold:
        verbose_print(verbose, 'STATUS:\t{} tanimoto threshold being used'.format(float(tanimoto_threshold)*100))
        cytosol_compartmentID = get_compartmentID_from_db(DB, 'cytosol')
        extracell_compartmentID = get_compartmentID_from_db(DB, 'extracellular')
        SIM = ss.TanimotoStructureSimilarity(R.targets, DB.get_all_compounds(),
                                             cytosol_compartmentID, extracell_compartmentID,
                                             verbose, tanimoto_threshold)
        OUTPUT.output_final_targets(SIM.finaltargets, tanimoto_threshold)
        if DB_missing_targets and len(SIM.finaltargets) == 1:
            verbose_print(verbose, 'STATUS:\tNo structurally similar compounds available with threshold %.2f' % tanimoto_threshold)
            return ([], R.ignorerxns, OUTPUT, temp_imgs_PATH)
        else:
          return(SIM.finaltargets, R.ignorerxns, OUTPUT, temp_imgs_PATH)
    else:
        return(R.targets, R.ignorerxns, OUTPUT, temp_imgs_PATH)

def retrieve_constraints(allrxns, allcpds, ignore_reactions, database):
    '''
    Generates database constraints or uses previously generated
    database constraints (.constraints) file
    '''
    DB = Q.Connector(database)

    def store_constraint_file(filename, LP):
        '''store generated constraints into .constraints'''
        
        print ('STATUS:\tDumping newly generated variables...')

        with open(filename, 'wb') as fout1:
            
            print ('STATUS:\tDumping LP structure...')
            pickle.dump(LP.lp, fout1)
            
            print ('STATUS:\tDumping all compounds...')
            pickle.dump(LP.allcpds, fout1)

            print ('STATUS:\tDumping all reaction variables...')
            pickle.dump(LP.variables, fout1)

            print ('STATUS:\tDumping all reactions 1...')
            pickle.dump(LP.allrxnsrev_dict_rev, fout1)
  
            print ('STATUS:\tDumping all reactions 2...')
            pickle.dump(LP.allrxnsrev_dict, fout1)

            print ('STATUS:\tDumping all reactions 3...')
            pickle.dump(LP.allrxnsrev, fout1)

    def unload_constraint_file(filename):
        '''unload constraints from a .constraints'''

        print ('STATUS:\tLoading pre-stored variables...')

        with open(filename, 'rb') as fin1:
            
            print ('STATUS:\tLoading LP structure...')
            lp = pickle.load(fin1)

            print ('STATUS:\tLoading all compounds...')
            allcompounds4matrix = pickle.load(fin1)

            print ('STATUS:\tLoading all reaction variables...')
            variables = pickle.load(fin1)

            print ('STATUS:\tLoading all reactions 1...')
            allrxnsrev_dict_rev = pickle.load(fin1)

            print ('STATUS:\tLoading all reactions 2...')
            allrxnsrev_dict = pickle.load(fin1)

            print ('STATUS:\tLoading all reactions 3...')
            allrxnsrev = pickle.load(fin1)

        return (lp, allcompounds4matrix, variables, allrxnsrev_dict_rev, allrxnsrev_dict, allrxnsrev)

    def load_preconstructed_constraint_files(media, mediatype, args):
        '''load defualt .constraint files'''

        print ('WARNING:\tNo database constraint file specified using pre constructed database constraint file for database')
        if evaluate_reactions =='all':

            (lp, allcompounds4matrix, variables, allrxnsrev_dict_rev, allrxnsrev_dict, allrxnsrev) = unload_constraint_file(PATH+'/ConstructedDatabases/' + DEFAULT_DB_NAME + '.constraints')
            
            return (lp, allcompounds4matrix, variables, allrxnsrev_dict_rev, allrxnsrev_dict, allrxnsrev)

        elif evaluate_reactions =='chem':

            (lp, allcompounds4matrix, variables, allrxnsrev_dict_rev, allrxnsrev_dict, allrxnsrev) = unload_constraint_file(PATH+'/ConstructedDatabases/' + DEFAULT_DB_NAME + '_chem.constraints')
            
            return (lp, allcompounds4matrix, variables, allrxnsrev_dict_rev, allrxnsrev_dict, allrxnsrev)  

        elif evaluate_reactions =='bio':

            (lp, allcompounds4matrix, variables, allrxnsrev_dict_rev, allrxnsrev_dict, allrxnsrev) = unload_constraint_file(PATH+'/ConstructedDatabases/' + DEFAULT_DB_NAME + '_bio.constraints')
            
            return (lp, allcompounds4matrix, variables, allrxnsrev_dict_rev, allrxnsrev_dict, allrxnsrev)  



    ###RETRIEVE SPECIFIED BY USER CONSTRAINTS###
    if generate_database_constraints:
        LP = co.ConstructInitialLP(allrxns, allcpds, DB, ignore_reactions)
        store_constraint_file(generate_database_constraints, LP)

    elif database_constraints:
        (lp, allcompounds4matrix, variables, allrxnsrev_dict_rev, allrxnsrev_dict, allrxnsrev) = unload_constraint_file(database_constraints) 
        LP = co.ConstructInitialLP(allrxns, allcompounds4matrix, DB,
                                   ignore_reactions, lp, variables, allrxnsrev_dict_rev,
                                   allrxnsrev_dict, allrxnsrev)
    else:
        if media_for_FBA=='Carbon-D-Glucose':
            
            (lp, allcompounds4matrix, variables, allrxnsrev_dict_rev, allrxnsrev_dict, allrxnsrev) = load_preconstructed_constraint_files(media_for_FBA, 'GL', args)
        
        elif media_for_FBA=='Complete':
            
            (lp, allcompounds4matrix, variables, allrxnsrev_dict_rev, allrxnsrev_dict, allrxnsrev) = load_preconstructed_constraint_files(media_for_FBA, 'CP', args)
        
            LP = co.ConstructInitialLP(allrxns, allcompounds4matrix, DB,
                                       ignore_reactions, lp, variables, allrxnsrev_dict_rev,
                                       allrxnsrev_dict, allrxnsrev)
        else: 
            print ('ERROR:\tNo identified pre constraint file...stopping run')

    return LP

def construct_and_run_integerprogram(targets, output, database):
    '''
    Constructs ILP and solves it identifying shortest path to the target
    '''

    DB = Q.Connector(database)

    if timer_output:

        IP = ip_pulp.IntergerProgram(DB, limit_reactions,
                                    limit_cycles, k_number_of_paths,
                                    cycles, verbose, solver_time_limit, output)
    else:

        IP = ip_pulp.IntergerProgram(DB, limit_reactions,
                                    limit_cycles, k_number_of_paths,
                                    cycles, verbose, solver_time_limit, timer_output)

    return IP

#Run functions
targets, ignore_reactions, output, temp_imgs_PATH = read_in_and_generate_output_files(database, targets_path, run_tanimoto_threshold)
LP = retrieve_constraints(all_db_reactions, all_db_compounds, ignore_reactions, database)
IP = construct_and_run_integerprogram(targets, output, database)


### Find pathways for target using the built or loaded database and constraint object

In [ ]:
#find pathway for targets 
def retrieve_shortestpath(target_info, IP, LP, database, output, temp_imgs_PATH, timer_output, media_for_FBA, 
                          flux_balance_analysis, knockouts, images, figures_graphviz, figures_chemdraw, evaluate_reactions,
                          show_rxn_info, output_path, multiple_solutions, start_compounds, gene_compatability,
                          orgs_gbs, gbs_orgs, RGC, keggorganisms_ids, output_genecompdb, verbose):
    '''Retrieve the shortest path for target organism'''

    start = timer()
    DB = Q.Connector(database)
    SP = None
    ranktype = None

    verbose_print(verbose, "STATUS: getting path for {}".format(target_info))

    if images == False:
        _images = False
    else:
        _images = True

    if not _specific_target(target_info[2]) and not start_compounds:
        print ('WARNING: No organism given therefore target {} compound will be skipped ... '.format(target_info[0]))

    else:

        if start_compounds:

            incpds_active = rtsc.readfile_startcompounds(start_compounds)
            inrxns_active = []

        else:

            incpds_active = DB.get_compounds_in_model(target_info[2])
            inrxns_active = DB.get_reactions_in_model(target_info[2])

        if target_info[0] in incpds_active: #Check if compound exists in organism

            output.output_compound_natively_present_in_target_organism(target_info)

        else:
            optimal_pathways = IP.run_glpk(LP, incpds_active, inrxns_active, target_info[0],
                                           multiplesolutions=multiple_solutions)
            if optimal_pathways:                    
                uniq_externalrxns = []
                for path in optimal_pathways:
                    path_org = []
                    for rxn in path:
                        rxn = re.sub('_F$', '', rxn)
                        rxn = re.sub('_R$', '', rxn)
                        path_org.append(rxn)
                    uniq_externalrxns.append(list(set(path_org) - set(inrxns_active)))

                ex_info = ei.Extract_Information(optimal_pathways, incpds_active, inrxns_active, DB)
                output.output_shortest_paths(target_info, ex_info.temp_rxns)

                R = rf.ReactionFiles(output_path, DB, ex_info.temp_rxns,
                                 target_info[0], target_info[2], incpds_active)

                output.output_raw_solutions(target_info[0], target_info[2], R.ordered_paths,
                                            ex_info.temp_rxns, ex_info.temp_external, incpds_active)

                if flux_balance_analysis:
                    opt_fba = run_flux_balance_analysis(target_info, ex_info,
                                                        incpds_active, inrxns_active,
                                                        media_for_FBA, knockouts,
                                                        output, DB, verbose)

                    R.generate_cdxml_files(RP=None, ranktype=None, fba_fluxes=opt_fba.fbasol.fluxes, show_rxn_info=show_rxn_info)
                    
                    if figures_graphviz:

                        G = spgd.GraphDot(DB, output_path, incpds_active, inrxns_active,
                                          temp_imgs_PATH, opt_fba.fbasol.fluxes)
                        G.sc_graph(target_info[0], target_info[2], ex_info.temp_rxns, _images)

                elif not flux_balance_analysis:
                    R.generate_cdxml_files()

                    if figures_graphviz:
                        G = spgd.GraphDot(DB, output_path, incpds_active, inrxns_active, temp_imgs_PATH)
                        G.sc_graph(target_info[0], target_info[2], ex_info.temp_rxns, _images)

                if gene_compatability:
                    verbose_print(verbose, 'STATUS:\tOptimizing gene sequences for optimal pathway reaction enzymes...')
                    enzymes = list(R.enzyme_set)
                    if len(enzymes) != 0:
                        output.generate_gc_directory()
                        for enzyme in enzymes:
                            if os.path.isfile(os.path.join(output.GC_output_path, "geneseqs_{}_{}.txt".format(enzyme, target_info[2]))):
                                print ("STATUS: already have sequence information for {} in {}".format(enzyme, target_info[2]))
                            else:
                                gce(enzyme, orgs_gbs, gbs_orgs, target_info[2], RGC, keggorganisms_ids, output_genecompdb,
                                    output_directory=output.GC_output_path, user_cai_table=user_cai_table,
                                    cai_optimal_threshold=cai_threshold)
                    else:
                        verbose_print(verbose, 'STATUS:')

            else:
                output.output_shortest_paths(target_info, [])
                if flux_balance_analysis:
                    verbose_print(verbose, 'WARNING:\tNo optimal path for %s in species %s therefore no flux balance will be performed' % (target_info[0], target_info[2]))
    end = timer()
    end = timer()

    if timer_output:

        output.output_timer('Time to find all paths for {}\t{}\t{}\n'.format(target_info[0], (end-start), (end-start)/60))

    verbose_print(verbose, "Time to find all paths for "+str(target_info[0])+' '+str(end - start))


##Run retrieve_shortestpath
if gene_compatability:
    orgs_gbs, gbs_orgs, R, keggorganisms_ids, output_genecompdb = gc(database, 
                                                                     output_directory=output_path,
                                                                     default_db='')
else:
    orgs_gbs=False
    gbs_orgs=False
    R=False
    keggorganisms_ids=False
    output_genecompdb=False

args_targets = [targets[i:i+processors]
              for i in range(0, len(targets), processors)]
for targets in args_targets:
    processes = []
    for target in targets:
        processes.append(Process(target=retrieve_shortestpath, args=(target, IP, LP, database, output,
                                                    temp_imgs_PATH, timer_output, media_for_FBA,
                                                    flux_balance_analysis, knockouts, images,
                                                    figures_graphviz, figures_chemdraw, evaluate_reactions, show_rxn_info,
                                                    output_path, multiple_solutions, start_compounds, gene_compatability,
                                                    orgs_gbs, gbs_orgs, R, keggorganisms_ids, output_genecompdb,
                                                    verbose)))

    for p in processes:        
        p.start()
    for p in processes:
        p.join()

if output_xlsx_format:
    output.convert_output_2_xlsx()

if output_html:
    print("STATUS: writing html file")
    gh.HtmlOutput(len(targets), output_path, flux_balance_analysis, figures_graphviz, database, output_path+"/Results.html")

'''Remove all temporary images'''
shutil.rmtree(temp_imgs_PATH)

'''Removes all dot files if they exist'''
for filename in glob.glob(output_path+"/solution_figures/*.dot"):
    os.remove(filename)

### Load in html file of results, only run this if option output_html is set to True

In [ ]:
HTML(output_path+"/Results.html")